In [9]:
%reload_ext autoreload
%autoreload 2
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import warnings
from itertools import product
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from tqdm.notebook import tqdm
from sklearn.exceptions import ConvergenceWarning
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

In [10]:
Xs=[]
Ys=[]
def load_data(data_name,N,M,K,pixel):
    for i in range(K):
        id=random.randint(0,K)
        img=cv2.imread(f'./training_data/{data_name}/numbers_{id}.png')
        numbers=np.load(f'./training_data/{data_name}/numbers_{id}.npy')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # plt.imshow(img, cmap='gray')
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # plt.imshow(img, cmap='gray')
        # print(img.shape)
        img_rs=cv2.resize(gray,(pixel*M,pixel*N),interpolation=cv2.INTER_AREA)
    # img_rs=cv2.bitwise_not(img_rs)
    # print(img_rs.shape,i)

        for i in range(N):
            for j in range(M):
                digit_square = img_rs[i*pixel:(i+1)*pixel, j*pixel:(j+1)*pixel]
                x_i=digit_square.reshape(1, -1)
                # print(x_i.shape,i,j)
                y=numbers[i,j]
                Xs.append(x_i)
                Ys.append(y)

In [11]:
# 白地のまま学習する

# 画像を読み込む
try:
    train
except:
    train=1
save=1
pixel=60
N1,M1=10,10
N2,M2=50,50
K1,K2=300,5

data_name='numbers_line_3'
load_data(data_name,N1,M1,100,pixel)
data_name='numbers_line'
load_data(data_name,N2,M2,5,pixel)
data_name='numbers'   
load_data(data_name,N2,M2,5,pixel)
data_name='numbers_ang'
load_data(data_name,N1,M1,100,pixel)
Xs=pd.DataFrame(np.array(Xs).reshape(-1,pixel*pixel))
Ys=pd.DataFrame(np.array(Ys).reshape(-1,1),columns=['label'])
print(Xs.shape,Ys.shape)
df=pd.concat([Ys,Xs],axis=1)
# df=df.loc[df.iloc[:,0]!=0]
df.shape


(45000, 3600) (45000, 1)


(45000, 3601)

In [12]:
X_org, y_org = df.iloc[:,1:], df.iloc[:,0]
X_org = np.array(X_org / 255.0, dtype="double")
y_org = np.array(y_org, dtype="uint8")
X, X_test, y, y_test = model_selection.train_test_split(
    X_org, y_org,test_size=0.2, shuffle=True,random_state=42
)
scaler = StandardScaler()
scaler.fit(X)

# データのスケーリング
X = scaler.transform(X)
X_test = scaler.transform(X_test)

In [13]:
# load
# scaler = pd.read_pickle('./pickle/Rand_numbers_line_3_scaler.pickle')
clf=pd.read_pickle('./pickle/Rand_numbers_line_3_clf.pickle')
clf2=pd.read_pickle('./pickle/rf_clf.pickle')
clf3=pd.read_pickle('./pickle/Rand_numbers_ang_clf.pickle')
clf_vote=VotingClassifier(estimators=[('rf_l3', clf), ('rf', clf2),("rf_ang",clf3)], voting='hard')
clf_vote.fit(X, y)

In [ ]:

acc_train = 100.0 * clf_vote.score(X, y)
print("logis_acc_train", acc_train)

# テストデータに対する識別精度の計算
acc_test = 100.0 * clf_vote.score(X_test, y_test)
print("logis_acc_test", acc_test)

logis_acc_train 100.0
logis_acc_test 94.24444444444444


In [ ]:
clf_vote.predict(X_test)

array([0, 0, 5, ..., 0, 9, 5], dtype=uint8)

In [ ]:
import pickle
with open('./pickle/ensemble3_clf.pickle', mode='wb') as fp:
    pickle.dump(clf_vote, fp)
with open('./pickle/ensemble3_scaler.pickle', mode='wb') as fp:
    pickle.dump(scaler, fp)